# Project Report: Smartsphere – An Agentic AI for Productivity
*Submitted by Aurosikha Mohanty*

In [1]:
!pip install gradio

# Import Packages

In [4]:
import gradio as gr

In [5]:
import pickle
import os.path
import openai
import re
from dotenv import load_dotenv
import os
import pkg_resources
print(pkg_resources.get_distribution("google-auth").version)
print(pkg_resources.get_distribution("google-auth-oauthlib").version)
from google_auth_oauthlib.flow import InstalledAppFlow
import google_auth_oauthlib
import google_auth_oauthlib.flow
print("google_auth_oauthlib location:", google_auth_oauthlib.__file__)
print("google_auth_oauthlib.flow.InstalledAppFlow exists:",
      hasattr(google_auth_oauthlib.flow, "InstalledAppFlow"))

from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

from email.mime.text import MIMEText
import base64
from googleapiclient.http import MediaFileUpload
import io
import docx  # pip install python-docx


import streamlit as st
import json
from datetime import datetime, timedelta, time, timezone


2.40.3
1.2.2
google_auth_oauthlib location: C:\Users\auros\anaconda3\Lib\site-packages\google_auth_oauthlib\__init__.py
google_auth_oauthlib.flow.InstalledAppFlow exists: True


# Configuration

## OPEN AI

In [8]:
load_dotenv()  # Automatically loads .env from current directory
api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=api_key)


In [9]:
def gpt4_chat(prompt, history=None, model_name="gpt-4o"):
    messages = []
    if history:
        messages += history  # Optionally include prior conversation turns
    messages.append({"role": "user", "content": prompt})
    response = client.chat.completions.create(
        model=model_name,
        messages=messages
    )
    return response.choices[0].message.content


## Google API 

In [11]:
SCOPES = [
    'https://www.googleapis.com/auth/calendar',
    'https://www.googleapis.com/auth/gmail.send',
    'https://www.googleapis.com/auth/gmail.readonly',  
    'https://www.googleapis.com/auth/tasks',
    'https://www.googleapis.com/auth/drive',
    'https://www.googleapis.com/auth/spreadsheets'
]
flow = InstalledAppFlow.from_client_secrets_file('Credentials_Google.json', SCOPES)
creds = flow.run_local_server(port=0)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=188845890770-e0qbo450370su6oenrcbvqvbsvc5aq8t.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A60141%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Ftasks+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets&state=oRKlAhgsJQM47bF4KnYUSPV1RYCv4m&access_type=offline


# Defining Tools

## Google Calendar 

In [14]:
def create_calendar_event(title, start_time, end_time, location="", attendees=None):
    service = build('calendar', 'v3', credentials=creds)
    event = {
        'summary': title,
        'location': location,
        'start': {'dateTime': start_time, 'timeZone': 'UTC'},
        'end': {'dateTime': end_time, 'timeZone': 'UTC'},
    }
    if attendees:
        event['attendees'] = [{'email': email.strip()} for email in attendees]
    event = service.events().insert(calendarId='primary', body=event, sendUpdates='all').execute()
    return f"Event '{title}' scheduled! View it here: {event.get('htmlLink')} (ID: {event['id']})"

def edit_calendar_event(event_id, title=None, start_time=None, end_time=None, location=None, attendees=None):
    service = build('calendar', 'v3', credentials=creds)
    event = service.events().get(calendarId='primary', eventId=event_id).execute()
    if title: event['summary'] = title
    if start_time: event['start']['dateTime'] = start_time
    if end_time: event['end']['dateTime'] = end_time
    if location: event['location'] = location
    if attendees: event['attendees'] = [{'email': email.strip()} for email in attendees]
    updated_event = service.events().update(calendarId='primary', eventId=event_id, body=event, sendUpdates='all').execute()
    return f"Event updated: {updated_event.get('htmlLink')}"

def cancel_calendar_event(event_id):
    service = build('calendar', 'v3', credentials=creds)
    service.events().delete(calendarId='primary', eventId=event_id, sendUpdates='all').execute()
    return f"Event {event_id} cancelled."

def search_calendar_events(query=None, start_date=None, end_date=None):
    service = build('calendar', 'v3', credentials=creds)
    time_min = start_date if start_date else None
    time_max = end_date if end_date else None
    events_result = service.events().list(
        calendarId='primary',
        timeMin=time_min,
        timeMax=time_max,
        q=query,
        singleEvents=True,
        orderBy='startTime',
        maxResults=10
    ).execute()
    events = events_result.get('items', [])
    if not events:
        return "No events found matching your criteria."
    return "\n".join([
        f"{i+1}. {e['summary']} at {e['start'].get('dateTime', e['start'].get('date'))} (ID: {e['id']})"
        for i, e in enumerate(events)
    ])


## GMAIL

In [16]:
def send_gmail(to, subject, body, style="formal"):
    # Optionally use LLM to adjust style
    service = build('gmail', 'v1', credentials=creds)
    message = MIMEText(body)
    message['to'] = to
    message['subject'] = subject
    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
    sent = service.users().messages().send(userId='me', body={'raw': raw}).execute()
    return f"Email sent to {to} with subject '{subject}'."

def search_gmail(query):
    service = build('gmail', 'v1', credentials=creds)
    results = service.users().messages().list(userId='me', q=query, maxResults=10).execute()
    messages = results.get('messages', [])
    email_list = []
    for msg in messages:
        msg_detail = service.users().messages().get(userId='me', id=msg['id'], format='metadata', metadataHeaders=['subject', 'from']).execute()
        headers = {h['name'].lower(): h['value'] for h in msg_detail['payload'].get('headers', [])}
        email_list.append({
            "id": msg['id'],
            "subject": headers.get('subject', 'No Subject'),
            "from": headers.get('from', 'Unknown Sender'),
            "snippet": msg_detail.get('snippet', '')
        })
    if not email_list:
        return "No emails found matching your query."
    formatted = "\n".join([f"{i+1}. From: {e['from']}, Subject: {e['subject']}, Snippet: {e['snippet']}" for i, e in enumerate(email_list)])
    return formatted

def search_gmail_smart(intent, max_results=10):
    service = build('gmail', 'v1', credentials=creds)
    gmail_query = gpt4_chat(f"Convert this user request to a Gmail search query (short, Gmail advanced search): '{intent}'.")
    if not gmail_query:
        gmail_query = intent
    results = service.users().messages().list(userId='me', q=gmail_query, maxResults=max_results).execute()
    messages = results.get('messages', [])
    if not messages:
        return "No emails found matching your request."
    summary_lines = []
    for msg in messages:
        msg_detail = service.users().messages().get(userId='me', id=msg['id'], format='full').execute()
        headers = {h['name'].lower(): h['value'] for h in msg_detail['payload'].get('headers', [])}
        sender = headers.get('from', 'Unknown Sender')
        subject = headers.get('subject', 'No Subject')
        snippet = msg_detail.get('snippet', '')
        text_parts = []
        if 'parts' in msg_detail['payload']:
            for part in msg_detail['payload']['parts']:
                if part['mimeType'] == 'text/plain' and part['body'].get('data'):
                    text_parts.append(base64.urlsafe_b64decode(part['body']['data']).decode('utf-8', errors='ignore'))
        else:
            if 'body' in msg_detail['payload'] and msg_detail['payload']['body'].get('data'):
                text_parts.append(base64.urlsafe_b64decode(msg_detail['payload']['body']['data']).decode('utf-8', errors='ignore'))
        msg_text = " ".join(text_parts) or snippet
        llm_summary = gpt4_chat(f"Summarize in 1-2 sentences what this email is about. Content:\n{msg_text[:2000]}")
        summary_lines.append(f"From: {sender}\nSubject: {subject}\nSummary: {llm_summary}\n")
    return "\n".join(summary_lines)
def count_gmail_smart(intent):
    service = build('gmail', 'v1', credentials=creds)
    # Use LLM to convert intent to Gmail query
    gmail_query = gpt4_chat(f"Convert this user request to a Gmail search query (Gmail advanced search syntax, as short as possible, no quotes): '{intent}'.")
    if not gmail_query:
        gmail_query = intent
    # You can use a higher maxResults if you want, but for large queries, Gmail may cap at 500
    results = service.users().messages().list(userId='me', q=gmail_query, maxResults=500).execute()
    messages = results.get('messages', [])
    count = len(messages)
    return f"Found {count} email(s) matching: {intent}."


## Google- Drive

In [18]:
def search_drive_files(query):
    service = build('drive', 'v3', credentials=creds)
    results = service.files().list(
        q=query,
        pageSize=10,
        fields="files(id, name, mimeType, webViewLink)"
    ).execute()
    files = results.get('files', [])
    if not files:
        return []
    return [
        {"id": f["id"], "name": f["name"], "type": f["mimeType"], "url": f["webViewLink"]}
        for f in files
    ]
def search_drive_files(query):
    from googleapiclient.discovery import build

    # If query does not use Drive API syntax, convert to "name contains '...'"
    if not any(x in query for x in ["name contains", "mimeType", "fullText contains", "'"]):
        # For multiple words, search for all terms (AND)
        terms = query.strip().split()
        query = " and ".join([f"name contains '{term}'" for term in terms])

    service = build('drive', 'v3', credentials=creds)
    results = service.files().list(
        q=query,
        pageSize=10,
        fields="files(id, name, mimeType, webViewLink)"
    ).execute()
    files = results.get('files', [])
    if not files:
        return []
    return [
        {
            "id": f["id"],
            "name": f["name"],
            "type": f["mimeType"],
            "url": f.get("webViewLink")
        }
        for f in files
    ]


In [19]:
def drive_file_action(action, file_id=None, file_path=None, folder_id=None, new_name=None, email=None, role='reader', download_path=None):
    service = build('drive', 'v3', credentials=creds)
    if action == 'upload':
        from googleapiclient.http import MediaFileUpload
        file_metadata = {'name': file_path.split('/')[-1]}
        media = MediaFileUpload(file_path, resumable=True)
        file = service.files().create(body=file_metadata, media_body=media, fields='id, webViewLink').execute()
        return f"File uploaded. View at: {file.get('webViewLink')}"
    elif action == 'download':
        from googleapiclient.http import MediaIoBaseDownload
        import io
        fh = io.FileIO(download_path, 'wb')
        request = service.files().get_media(fileId=file_id)
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
        fh.close()
        return f"File downloaded to {download_path}"
    elif action == 'delete':
        service.files().delete(fileId=file_id).execute()
        return "File deleted."
    elif action == 'rename':
        file = service.files().update(fileId=file_id, body={'name': new_name}).execute()
        return f"File renamed to {new_name}."
    elif action == 'move':
        # Requires removing from current parents and adding to new
        file = service.files().get(fileId=file_id, fields='parents').execute()
        previous_parents = ",".join(file.get('parents', []))
        service.files().update(fileId=file_id, addParents=folder_id, removeParents=previous_parents, fields='id, parents').execute()
        return f"File moved to folder {folder_id}."
    elif action == 'share':
        permission = {
            'type': 'user',
            'role': role,
            'emailAddress': email
        }
        service.permissions().create(fileId=file_id, body=permission, fields='id').execute()
        return f"File shared with {email} as {role}."
    elif action == 'details':
        file = service.files().get(fileId=file_id, fields='id, name, mimeType, owners, modifiedTime, webViewLink').execute()
        return str(file)
    else:
        return "Unsupported action."


## Google Documents

In [21]:

def read_docx_from_drive(file_id):
    service = build('drive', 'v3', credentials=creds)
    request = service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    from googleapiclient.http import MediaIoBaseDownload
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while not done:
        status, done = downloader.next_chunk()
    fh.seek(0)
    doc = docx.Document(fh)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)


def edit_docx_on_drive(file_id, new_text, new_file_name=None):
    # Download the doc
    service = build('drive', 'v3', credentials=creds)

    # Download as docx
    request = service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    from googleapiclient.http import MediaIoBaseDownload
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while not done:
        status, done = downloader.next_chunk()
    fh.seek(0)

    # Edit with python-docx
    doc = docx.Document()
    for line in new_text.split('\n'):
        doc.add_paragraph(line)
    # Save to temp file
    temp_file = "temp_edit.docx"
    doc.save(temp_file)

    # Upload back to Drive (optionally, new name)
    file_metadata = {'name': new_file_name or "Edited_Doc.docx"}
    media = MediaFileUpload(temp_file, mimetype='application/vnd.openxmlformats-officedocument.wordprocessingml.document')
    updated = service.files().create(body=file_metadata, media_body=media, fields='id, webViewLink').execute()
    return updated.get('webViewLink')

def create_google_doc(title, content):
    from googleapiclient.discovery import build
    service = build('docs', 'v1', credentials=creds)
    doc = service.documents().create(body={"title": title}).execute()
    doc_id = doc.get('documentId')
    # Add content
    requests = [{"insertText": {"location": {"index": 1}, "text": content}}]
    service.documents().batchUpdate(documentId=doc_id, body={"requests": requests}).execute()
    return f"https://docs.google.com/document/d/{doc_id}/edit"

def append_to_sheet(spreadsheet_id, range_name, values):
    service = build('sheets', 'v4', credentials=creds)
    body = {'values': [values]}
    service.spreadsheets().values().append(
        spreadsheetId=spreadsheet_id,
        range=range_name,
        valueInputOption="USER_ENTERED",
        body=body
    ).execute()
    return f"Added row to sheet {spreadsheet_id}: {values}"

In [22]:
import PyPDF2

def read_pdf_from_drive(file_id):
    # Download PDF using Google Drive API (pseudo, needs your logic)
    local_pdf_path = download_drive_file(file_id)
    with open(local_pdf_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        text = "\n".join(page.extract_text() for page in reader.pages if page.extract_text())
    return text[:5000]  # return only first 5000 chars for LLM context


In [23]:
def summarize_text(text, max_words=200):
    # Calls your LLM to summarize
    prompt = f"Summarize the following text in under {max_words} words:\n\n{text}"
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content


## Google Task - Planner

In [25]:
def add_google_task(title, due=None, notes=""):
    service = build('tasks', 'v1', credentials=creds)
    task = {'title': title}
    if due:
        task['due'] = due  # 'YYYY-MM-DDTHH:MM:SSZ'
    if notes:
        task['notes'] = notes
    result = service.tasks().insert(tasklist='@default', body=task).execute()
    return f"Task '{title}' added to your Google Tasks."

# GLOBAL VARIABLE TO STORE THE LAST PLAN
last_planned_schedule = []

def get_calendar_busy_slots_for_next_week(days=7):
    """
    Returns a list of busy time intervals (as tuples) for the next `days` days from Google Calendar.
    """
    from googleapiclient.discovery import build
    service = build('calendar', 'v3', credentials=creds)
    now = datetime.now(timezone.utc)
    start = now.isoformat()
    end = (now + timedelta(days=days)).isoformat()
    events_result = service.events().list(
        calendarId='primary',
        timeMin=start,
        timeMax=end,
        singleEvents=True,
        orderBy='startTime'
    ).execute()
    events = events_result.get('items', [])
    busy = []
    for event in events:
        st = event['start'].get('dateTime', event['start'].get('date'))
        et = event['end'].get('dateTime', event['end'].get('date'))
        busy.append((st, et))
    return busy

def infer_free_slots_from_calendar(busy_slots, days=7, work_hours=(8, 22), slot_minutes=60):
    """
    Returns free slots as a list of datetimes (RFC3339 strings) for the next `days` days.
    Only considers slots between work_hours (in 24hr, e.g. 8 to 22).
    """
    now = datetime.now(timezone.utc)
    free_slots = []
    all_busy = []
    # Flatten busy slots into datetime objects
    for st, et in busy_slots:
        try:
            st_dt = datetime.fromisoformat(st.replace('Z','+00:00')) if 'T' in st else datetime.fromisoformat(st)
            et_dt = datetime.fromisoformat(et.replace('Z','+00:00')) if 'T' in et else datetime.fromisoformat(et)
            all_busy.append((st_dt, et_dt))
        except Exception as e:
            continue

    for day in range(days):
        cur = now + timedelta(days=day)
        d = cur.date()
        wh_start = datetime.combine(d, time(work_hours[0], 0, tzinfo=timezone.utc))
        wh_end   = datetime.combine(d, time(work_hours[1], 0, tzinfo=timezone.utc))
        t = wh_start
        while t + timedelta(minutes=slot_minutes) <= wh_end:
            overlap = False
            for b_start, b_end in all_busy:
                # If this slot overlaps with a busy interval, skip
                if b_start < t + timedelta(minutes=slot_minutes) and b_end > t:
                    overlap = True
                    break
            if not overlap and t > now:
                free_slots.append(t.isoformat())
            t += timedelta(minutes=slot_minutes)
    return free_slots


# GLOBAL variable for last plan
last_planned_schedule = []
def plan_tasks_around_calendar(goal, topic, duration_days=30, subtasks=None):
    global last_planned_schedule
    # 1. Get subtasks
    if not subtasks or len(subtasks) == 0:
        prompt = (
            f"Break the goal '{goal}' (topic: {topic}) into 4-15 specific, actionable tasks for the next {duration_days} days. "
            "Return as a numbered list."
        )
        raw = gpt4_chat(prompt)  # Assumes you have this helper using GPT-4o!
        subtasks = [line.strip() for line in raw.split('\n') if line.strip() and line[0].isdigit()]

    # 2. Get busy/free slots (still as a list of RFC3339 strings)
    busy_slots = get_calendar_busy_slots_for_next_week(days=duration_days)
    free_slots = infer_free_slots_from_calendar(busy_slots, days=duration_days)
    # Group free slots by date (isoformat date only)
    from collections import defaultdict
    slot_by_day = defaultdict(list)
    for slot in free_slots:
        date_part = slot.split("T")[0]
        slot_by_day[date_part].append(slot)
    unique_days = sorted(slot_by_day.keys())

    # 3. Assign tasks spaced across unique days
    plan = []
    day_idx = 0
    for subtask in subtasks:
        if day_idx < len(unique_days):
            slot = slot_by_day[unique_days[day_idx]][0]
            plan.append({"task": subtask, "slot": slot})
            day_idx += 1
        else:
            # If we run out of days, reuse the last one or say "no slot"
            plan.append({"task": subtask, "slot": "no free slot found"})

    last_planned_schedule = plan.copy()  # Save for later user queries

    # 4. Show plan before scheduling
    print("\nAssistant: Here’s your suggested plan:")
    for idx, step in enumerate(plan, 1):
        print(f"{idx}. {step['task']} @ {step['slot']}")
    confirm = input("Assign these as Google Tasks with due dates? (yes/no): ").strip().lower()
    if confirm == "yes":
        for step in plan:
            if "no free slot" not in step['slot']:
                add_google_task(title=step['task'], due=step['slot'], notes=f"{topic} study plan")
            else:
                add_google_task(title=step['task'], notes=f"{topic} study plan (no date assigned)")
        return "Tasks scheduled based on your calendar!"
    else:
        return "Task planning cancelled."

# --- Utility for showing last plan ---
def show_last_planned_schedule():
    global last_planned_schedule
    if last_planned_schedule:
        print("Assistant: Here is the most recent planned schedule:")
        for idx, step in enumerate(last_planned_schedule, 1):
            print(f"{idx}. {step['task']} @ {step['slot']}")
        return last_planned_schedule
    else:
        print("Assistant: No plan has been created yet in this session.")
        return []

def delete_google_task(task_id, tasklist_id='@default'):
    """
    Delete a Google Task by its task_id.
    """
    from googleapiclient.discovery import build
    service = build('tasks', 'v1', credentials=creds)
    service.tasks().delete(tasklist=tasklist_id, task=task_id).execute()
    return f"Task {task_id} deleted."
def edit_google_task(task_id, title=None, notes=None, due=None, status=None, tasklist_id='@default'):
    """
    Edit a Google Task by its task_id. Only updates provided fields.
    """
    from googleapiclient.discovery import build
    service = build('tasks', 'v1', credentials=creds)
    task = service.tasks().get(tasklist=tasklist_id, task=task_id).execute()
    if title: task['title'] = title
    if notes: task['notes'] = notes
    if due: task['due'] = due
    if status: task['status'] = status  # e.g. 'completed', 'needsAction'
    updated_task = service.tasks().update(tasklist=tasklist_id, task=task_id, body=task).execute()
    return f"Task {task_id} updated."
def search_google_tasks(query=None, tasklist_id='@default'):
    """
    List (and search/filter) Google Tasks by title/notes content.
    If query is provided, returns only tasks containing query in title or notes.
    """
    from googleapiclient.discovery import build
    service = build('tasks', 'v1', credentials=creds)
    results = service.tasks().list(tasklist=tasklist_id, showCompleted=True, showHidden=True, maxResults=100).execute()
    tasks = results.get('items', [])
    if query:
        # Case-insensitive search in title or notes
        filtered = [
            t for t in tasks
            if query.lower() in t.get('title', '').lower() or query.lower() in t.get('notes', '').lower()
        ]
    else:
        filtered = tasks
    # Nicely format results
    if not filtered:
        return "No tasks found."
    summary = []
    for t in filtered:
        status = t.get('status', 'needsAction')
        due = t.get('due', 'No due date')
        summary.append(f"- [{status}] {t.get('title', 'Untitled')} (Due: {due}, ID: {t['id']})")
    return "\n".join(summary)


In [26]:
def read_gdoc_from_drive(file_id):
    """
    Reads all plain text from a Google Doc (native, not DOCX) in Google Drive.

    Args:
        file_id (str): Google Drive file ID of the Google Doc.

    Returns:
        str: The text content of the Google Doc.
    """
    try:
        from googleapiclient.discovery import build
        service = build('drive', 'v3', credentials=creds)
        # Export Google Doc as plain text
        response = service.files().export(fileId=file_id, mimeType='text/plain').execute()
        if isinstance(response, bytes):
            response = response.decode('utf-8')
        return response
    except Exception as e:
        print(f"Error reading Google Doc: {e}")
        return f"Error reading Google Doc: {e}"


# Final Execution

In [28]:
tools = [
   
    # -- Add/Create Event --
    {
        "type": "function",
        "function": {
            "name": "create_calendar_event",
            "description": "Creates a new Google Calendar event and invites attendees.",
            "parameters": {
                "type": "object",
                "properties": {
                    "title": {"type": "string", "description": "Event title"},
                    "start_time": {"type": "string", "description": "Start time (RFC3339 format)"},
                    "end_time": {"type": "string", "description": "End time (RFC3339 format)"},
                    "location": {"type": "string", "description": "Event location"},
                    "attendees": {"type": "array", "items": {"type": "string"}, "description": "List of attendee emails"}
                },
                "required": ["title", "start_time", "end_time"]
            }
        }
    },
    # -- Edit/Update Event --
    {
        "type": "function",
        "function": {
            "name": "edit_calendar_event",
            "description": "Edits an existing calendar event given its event ID.",
            "parameters": {
                "type": "object",
                "properties": {
                    "event_id": {"type": "string", "description": "Google Calendar event ID to update"},
                    "title": {"type": "string", "description": "Updated title (optional)"},
                    "start_time": {"type": "string", "description": "Updated start time (RFC3339, optional)"},
                    "end_time": {"type": "string", "description": "Updated end time (RFC3339, optional)"},
                    "location": {"type": "string", "description": "Updated location (optional)"},
                    "attendees": {"type": "array", "items": {"type": "string"}, "description": "Updated attendee emails (optional)"}
                },
                "required": ["event_id"]
            }
        }
    },
    # -- Cancel/Delete Event --
    {
        "type": "function",
        "function": {
            "name": "cancel_calendar_event",
            "description": "Deletes or cancels a Google Calendar event by event ID.",
            "parameters": {
                "type": "object",
                "properties": {
                    "event_id": {"type": "string", "description": "ID of the event to delete/cancel"}
                },
                "required": ["event_id"]
            }
        }
    },
    # -- Search/List Events --
    {
        "type": "function",
        "function": {
            "name": "search_calendar_events",
            "description": "Lists calendar events based on a date range, attendee, or keyword.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "Free-text keyword or attendee email (optional)"},
                    "start_date": {"type": "string", "description": "Start date (RFC3339, optional)"},
                    "end_date": {"type": "string", "description": "End date (RFC3339, optional)"}
                }
            }
        }
    },
  
      # --- Smart search by intent ---
    {
        "type": "function",
        "function": {
            "name": "search_gmail_smart",
            "description": "Searches Gmail for emails matching a topic or intent. Returns sender, subject, and a summary for each.",
            "parameters": {
                "type": "object",
                "properties": {
                    "intent": {"type": "string", "description": "What kind of emails to search, e.g. 'sent to recruiters', 'rejection emails', 'interview follow ups'."},
                    "max_results": {"type": "integer", "description": "Max emails to return", "default": 10}
                },
                "required": ["intent"]
            }
        }
    },
    # --- Classic search by Gmail query ---
    {
        "type": "function",
        "function": {
            "name": "search_gmail",
            "description": "Searches Gmail using a standard Gmail search query (e.g., 'subject:offer').",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "Gmail search query"}
                },
                "required": ["query"]
            }
        }
    },
    # --- Send Gmail ---
    {
        "type": "function",
        "function": {
            "name": "send_gmail",
            "description": "Sends an email via Gmail. Checks for formal/informal style, and confirms before sending.",
            "parameters": {
                "type": "object",
                "properties": {
                    "to": {"type": "string", "description": "Recipient"},
                    "subject": {"type": "string", "description": "Subject"},
                    "body": {"type": "string", "description": "Email content"},
                    "style": {"type": "string", "enum": ["formal", "informal"], "description": "Tone of email"}
                },
                "required": ["to", "subject", "body"]
            }
        }
    },
    # ---- Count emails lsit-----
    {
        "type": "function",
        "function": {
            "name": "count_gmail_smart",
            "description": "Counts how many Gmail emails match a given intent or natural-language query (e.g., 'rejection emails', 'offers from Amazon', etc).",
            "parameters": {
                "type": "object",
                "properties": {
                    "intent": {"type": "string", "description": "What kind of emails to count, e.g. 'rejection emails', 'offers', 'sent to recruiters', etc."}
                },
                "required": ["intent"]
            }
        }
    },
# Google tasks
    {
        "type": "function",
        "function": {
            "name": "add_google_task",
            "description": "Adds a task or to-do item to your Google Tasks.",
            "parameters": {
                "type": "object",
                "properties": {
                    "title": {"type": "string", "description": "Task title"},
                    "due": {"type": "string", "description": "Due date/time in RFC3339 format, optional"},
                    "notes": {"type": "string", "description": "Additional notes, optional"}
                },
                "required": ["title"]
            }
        }
    },
    {
    "type": "function",
    "function": {
        "name": "delete_google_task",
        "description": "Deletes a Google Task by its task ID.",
        "parameters": {
            "type": "object",
            "properties": {
                "task_id": {"type": "string", "description": "The Google Task's unique ID"}
            },
            "required": ["task_id"]
        }
    }
},
{
    "type": "function",
    "function": {
        "name": "edit_google_task",
        "description": "Edits a Google Task by task ID. Only updates provided fields.",
        "parameters": {
            "type": "object",
            "properties": {
                "task_id": {"type": "string", "description": "The Google Task's unique ID"},
                "title": {"type": "string", "description": "New task title (optional)"},
                "notes": {"type": "string", "description": "New notes (optional)"},
                "due": {"type": "string", "description": "New due date/time (optional)"},
                "status": {"type": "string", "description": "New status, e.g., 'completed' (optional)"}
            },
            "required": ["task_id"]
        }
    }
},
{
    "type": "function",
    "function": {
        "name": "search_google_tasks",
        "description": "Lists or searches Google Tasks by title/notes.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {"type": "string", "description": "Query string to filter tasks (optional)"}
            }
        }
    }
},
    {
    "type": "function",
    "function": {
        "name": "append_to_sheet",
        "description": "Appends a row to a Google Sheet.",
        "parameters": {
            "type": "object",
            "properties": {
                "spreadsheet_id": {"type": "string", "description": "Sheet ID"},
                "range_name": {"type": "string", "description": "Range (e.g., 'Sheet1!A1')"},
                "values": {"type": "array", "items": {"type": "string"}, "description": "Row values as a list"}
            },
            "required": ["spreadsheet_id", "range_name", "values"]
        }
    }
    },
    {
        "type": "function",
        "function": {
            "name": "search_drive_files",
            "description": "Searches your Google Drive for files matching a query string.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "Drive search query, e.g., name contains 'essay' or mimeType='application/pdf'"}
                },
                "required": ["query"]
            }
        }
    },
   {
    "type": "function",
    "function": {
        "name": "drive_file_action",
        "description": "Performs various Google Drive file actions (upload, download, delete, rename, move, share, get details).",
        "parameters": {
            "type": "object",
            "properties": {
                "action": {"type": "string", "enum": ["upload", "download", "delete", "rename", "move", "share", "details"], "description": "The Drive file operation to perform."},
                "file_id": {"type": "string", "description": "The Drive file's ID (if applicable)."},
                "file_path": {"type": "string", "description": "Local file path (for upload)."},
                "folder_id": {"type": "string", "description": "Drive folder ID (for move)."},
                "new_name": {"type": "string", "description": "New file name (for rename)."},
                "email": {"type": "string", "description": "Email to share with (for share)."},
                "role": {"type": "string", "description": "Share role: reader, writer, etc. (for share)."},
                "download_path": {"type": "string", "description": "Local path to save file (for download)."}
            },
            "required": ["action"]
        }
    }
},
    {
    "type": "function",
    "function": {
        "name": "plan_tasks_around_calendar",
        "description": "Breaks a goal (e.g., study prep) into subtasks, checks your Google Calendar for free slots, and schedules tasks over a time horizon.",
        "parameters": {
            "type": "object",
            "properties": {
                "goal": {"type": "string", "description": "The user's overall study/project goal."},
                "topic": {"type": "string", "description": "Topic to focus on, e.g., 'machine learning interview'."},
                "duration_days": {"type": "integer", "description": "Number of days to plan (e.g., 7 for one week)."},
                "subtasks": {"type": "array", "items": {"type": "string"}, "description": "A list of subtasks to schedule (optional; if blank, the LLM should propose them)."}
            },
            "required": ["goal", "topic", "duration_days"]
        }
    }
},
# Google doc
{
    "type": "function",
    "function": {
        "name": "read_docx_from_drive",
        "description": "Reads all text content from a DOCX file in Google Drive by file ID.",
        "parameters": {
            "type": "object",
            "properties": {
                "file_id": {"type": "string", "description": "Google Drive file ID"},
            },
            "required": ["file_id"]
        }
    }
},
{
    "type": "function",
    "function": {
        "name": "edit_docx_on_drive",
        "description": "Edits or overwrites a DOCX in Drive (by file ID) with new text, and optionally saves as a new name.",
        "parameters": {
            "type": "object",
            "properties": {
                "file_id": {"type": "string", "description": "Google Drive file ID"},
                "new_text": {"type": "string", "description": "Replacement text for the DOCX"},
                "new_file_name": {"type": "string", "description": "New file name (optional)"}
            },
            "required": ["file_id", "new_text"]
        }
    }
},
{
    "type": "function",
    "function": {
        "name": "create_google_doc",
        "description": "Creates a new Google Doc with the given title and text content.",
        "parameters": {
            "type": "object",
            "properties": {
                "title": {"type": "string", "description": "Title for the new doc"},
                "content": {"type": "string", "description": "Text content"}
            },
            "required": ["title", "content"]
        }
    }
},

   
    # -- Add/Create Event --
    {
        "type": "function",
        "function": {
            "name": "create_calendar_event",
            "description": "Creates a new Google Calendar event and invites attendees.",
            "parameters": {
                "type": "object",
                "properties": {
                    "title": {"type": "string", "description": "Event title"},
                    "start_time": {"type": "string", "description": "Start time (RFC3339 format)"},
                    "end_time": {"type": "string", "description": "End time (RFC3339 format)"},
                    "location": {"type": "string", "description": "Event location"},
                    "attendees": {"type": "array", "items": {"type": "string"}, "description": "List of attendee emails"}
                },
                "required": ["title", "start_time", "end_time"]
            }
        }
    },
    # -- Edit/Update Event --
    {
        "type": "function",
        "function": {
            "name": "edit_calendar_event",
            "description": "Edits an existing calendar event given its event ID.",
            "parameters": {
                "type": "object",
                "properties": {
                    "event_id": {"type": "string", "description": "Google Calendar event ID to update"},
                    "title": {"type": "string", "description": "Updated title (optional)"},
                    "start_time": {"type": "string", "description": "Updated start time (RFC3339, optional)"},
                    "end_time": {"type": "string", "description": "Updated end time (RFC3339, optional)"},
                    "location": {"type": "string", "description": "Updated location (optional)"},
                    "attendees": {"type": "array", "items": {"type": "string"}, "description": "Updated attendee emails (optional)"}
                },
                "required": ["event_id"]
            }
        }
    },
    # -- Cancel/Delete Event --
    {
        "type": "function",
        "function": {
            "name": "cancel_calendar_event",
            "description": "Deletes or cancels a Google Calendar event by event ID.",
            "parameters": {
                "type": "object",
                "properties": {
                    "event_id": {"type": "string", "description": "ID of the event to delete/cancel"}
                },
                "required": ["event_id"]
            }
        }
    },
    # -- Search/List Events --
    {
        "type": "function",
        "function": {
            "name": "search_calendar_events",
            "description": "Lists calendar events based on a date range, attendee, or keyword.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "Free-text keyword or attendee email (optional)"},
                    "start_date": {"type": "string", "description": "Start date (RFC3339, optional)"},
                    "end_date": {"type": "string", "description": "End date (RFC3339, optional)"}
                }
            }
        }
    },
  
      # --- Smart search by intent ---
    {
        "type": "function",
        "function": {
            "name": "search_gmail_smart",
            "description": "Searches Gmail for emails matching a topic or intent. Returns sender, subject, and a summary for each.",
            "parameters": {
                "type": "object",
                "properties": {
                    "intent": {"type": "string", "description": "What kind of emails to search, e.g. 'sent to recruiters', 'rejection emails', 'interview follow ups'."},
                    "max_results": {"type": "integer", "description": "Max emails to return", "default": 10}
                },
                "required": ["intent"]
            }
        }
    },
    # --- Classic search by Gmail query ---
    {
        "type": "function",
        "function": {
            "name": "search_gmail",
            "description": "Searches Gmail using a standard Gmail search query (e.g., 'subject:offer').",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "Gmail search query"}
                },
                "required": ["query"]
            }
        }
    },
    # --- Send Gmail ---
    {
        "type": "function",
        "function": {
            "name": "send_gmail",
            "description": "Sends an email via Gmail. Checks for formal/informal style, and confirms before sending.",
            "parameters": {
                "type": "object",
                "properties": {
                    "to": {"type": "string", "description": "Recipient"},
                    "subject": {"type": "string", "description": "Subject"},
                    "body": {"type": "string", "description": "Email content"},
                    "style": {"type": "string", "enum": ["formal", "informal"], "description": "Tone of email"}
                },
                "required": ["to", "subject", "body"]
            }
        }
    },
    # ---- Count emails lsit-----
    {
        "type": "function",
        "function": {
            "name": "count_gmail_smart",
            "description": "Counts how many Gmail emails match a given intent or natural-language query (e.g., 'rejection emails', 'offers from Amazon', etc).",
            "parameters": {
                "type": "object",
                "properties": {
                    "intent": {"type": "string", "description": "What kind of emails to count, e.g. 'rejection emails', 'offers', 'sent to recruiters', etc."}
                },
                "required": ["intent"]
            }
        }
    },
# Google tasks
    {
        "type": "function",
        "function": {
            "name": "add_google_task",
            "description": "Adds a task or to-do item to your Google Tasks.",
            "parameters": {
                "type": "object",
                "properties": {
                    "title": {"type": "string", "description": "Task title"},
                    "due": {"type": "string", "description": "Due date/time in RFC3339 format, optional"},
                    "notes": {"type": "string", "description": "Additional notes, optional"}
                },
                "required": ["title"]
            }
        }
    },
    {
    "type": "function",
    "function": {
        "name": "delete_google_task",
        "description": "Deletes a Google Task by its task ID.",
        "parameters": {
            "type": "object",
            "properties": {
                "task_id": {"type": "string", "description": "The Google Task's unique ID"}
            },
            "required": ["task_id"]
        }
    }
},
{
    "type": "function",
    "function": {
        "name": "edit_google_task",
        "description": "Edits a Google Task by task ID. Only updates provided fields.",
        "parameters": {
            "type": "object",
            "properties": {
                "task_id": {"type": "string", "description": "The Google Task's unique ID"},
                "title": {"type": "string", "description": "New task title (optional)"},
                "notes": {"type": "string", "description": "New notes (optional)"},
                "due": {"type": "string", "description": "New due date/time (optional)"},
                "status": {"type": "string", "description": "New status, e.g., 'completed' (optional)"}
            },
            "required": ["task_id"]
        }
    }
},
{
    "type": "function",
    "function": {
        "name": "search_google_tasks",
        "description": "Lists or searches Google Tasks by title/notes.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {"type": "string", "description": "Query string to filter tasks (optional)"}
            }
        }
    }
},
    {
    "type": "function",
    "function": {
        "name": "append_to_sheet",
        "description": "Appends a row to a Google Sheet.",
        "parameters": {
            "type": "object",
            "properties": {
                "spreadsheet_id": {"type": "string", "description": "Sheet ID"},
                "range_name": {"type": "string", "description": "Range (e.g., 'Sheet1!A1')"},
                "values": {"type": "array", "items": {"type": "string"}, "description": "Row values as a list"}
            },
            "required": ["spreadsheet_id", "range_name", "values"]
        }
    }
    },
    {
        "type": "function",
        "function": {
            "name": "search_drive_files",
            "description": "Searches your Google Drive for files matching a query string.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "Drive search query, e.g., name contains 'essay' or mimeType='application/pdf'"}
                },
                "required": ["query"]
            }
        }
    },
   {
    "type": "function",
    "function": {
        "name": "drive_file_action",
        "description": "Performs various Google Drive file actions (upload, download, delete, rename, move, share, get details).",
        "parameters": {
            "type": "object",
            "properties": {
                "action": {"type": "string", "enum": ["upload", "download", "delete", "rename", "move", "share", "details"], "description": "The Drive file operation to perform."},
                "file_id": {"type": "string", "description": "The Drive file's ID (if applicable)."},
                "file_path": {"type": "string", "description": "Local file path (for upload)."},
                "folder_id": {"type": "string", "description": "Drive folder ID (for move)."},
                "new_name": {"type": "string", "description": "New file name (for rename)."},
                "email": {"type": "string", "description": "Email to share with (for share)."},
                "role": {"type": "string", "description": "Share role: reader, writer, etc. (for share)."},
                "download_path": {"type": "string", "description": "Local path to save file (for download)."}
            },
            "required": ["action"]
        }
    }
},
    {
    "type": "function",
    "function": {
        "name": "plan_tasks_around_calendar",
        "description": "Breaks a goal (e.g., study prep) into subtasks, checks your Google Calendar for free slots, and schedules tasks over a time horizon.",
        "parameters": {
            "type": "object",
            "properties": {
                "goal": {"type": "string", "description": "The user's overall study/project goal."},
                "topic": {"type": "string", "description": "Topic to focus on, e.g., 'machine learning interview'."},
                "duration_days": {"type": "integer", "description": "Number of days to plan (e.g., 7 for one week)."},
                "subtasks": {"type": "array", "items": {"type": "string"}, "description": "A list of subtasks to schedule (optional; if blank, the LLM should propose them)."}
            },
            "required": ["goal", "topic", "duration_days"]
        }
    }
},
# Google doc
{
    "type": "function",
    "function": {
        "name": "read_docx_from_drive",
        "description": "Reads all text content from a DOCX file in Google Drive by file ID.",
        "parameters": {
            "type": "object",
            "properties": {
                "file_id": {"type": "string", "description": "Google Drive file ID"},
            },
            "required": ["file_id"]
        }
    }
},
{
    "type": "function",
    "function": {
        "name": "edit_docx_on_drive",
        "description": "Edits or overwrites a DOCX in Drive (by file ID) with new text, and optionally saves as a new name.",
        "parameters": {
            "type": "object",
            "properties": {
                "file_id": {"type": "string", "description": "Google Drive file ID"},
                "new_text": {"type": "string", "description": "Replacement text for the DOCX"},
                "new_file_name": {"type": "string", "description": "New file name (optional)"}
            },
            "required": ["file_id", "new_text"]
        }
    }
},
{
    "type": "function",
    "function": {
        "name": "create_google_doc",
        "description": "Creates a new Google Doc with the given title and text content.",
        "parameters": {
            "type": "object",
            "properties": {
                "title": {"type": "string", "description": "Title for the new doc"},
                "content": {"type": "string", "description": "Text content"}
            },
            "required": ["title", "content"]
        }
    }
},
{
    "type": "function",
    "function": {
        "name": "summarize_text",
        "description": "Summarizes a block of text, e.g., from a PDF, Google Doc, or pasted input.",
        "parameters": {
            "type": "object",
            "properties": {
                "text": {"type": "string", "description": "Text to summarize"},
                "max_words": {"type": "integer", "description": "Maximum words for the summary (optional, defaults to 200)"}
            },
            "required": ["text"]
        }
    }
},
    
    
{
    "type": "function",
    "function": {
        "name": "read_gdoc_from_drive",
        "description": "Reads all text content from a native Google Doc in Drive by file ID.",
        "parameters": {
            "type": "object",
            "properties": {
                "file_id": {"type": "string", "description": "Google Doc file ID"}
            },
            "required": ["file_id"]
        }
    }
}
,
{
    "type": "function",
    "function": {
        "name": "read_pdf_from_drive",
        "description": "Extracts all text content from a PDF in Google Drive using its file ID.",
        "parameters": {
            "type": "object",
            "properties": {
                "file_id": {"type": "string", "description": "Google Drive file ID of the PDF"}
            },
            "required": ["file_id"]
        }
    }
}

        # Add more tool schemas here (tasks, drive, sheets, etc.)
]


In [29]:
def get_confirmation(prompt="Are you sure? (yes/no): "):
    while True:
        response = input(prompt).strip().lower()
        if response in ["yes"]:
            return True
        if response in ["no","cancel"]:
            return False
        if response in ["quit", "exit", "esc", ]:
            break

        print("Assistant: Please type 'yes' to confirm or 'no' to cancel.")


In [30]:
from datetime import datetime

def get_today_string():
    # Example format: "Monday, July 1, 2025"
    return datetime.now().strftime("%A, %B %d, %Y")

def make_system_prompt():
    today = get_today_string()
    return (
        f"Today is {today}. "
        "You are a smart, agentic AI assistant with access to tools for Google Calendar, Gmail, Tasks, and Drive. "
        "For simple requests, just collect any missing info, confirm risky actions, and execute. "
        "For complex multi-step goals (like project or study planning), break it into a natural plan and guide the user step-by-step, but do not show subtasks unless the request is complex. "
        "Always be clear, concise, and conversational. Only ask for confirmation before sending, deleting, or sharing things."
        "When the user mentions 'today', always interpret it as the current system date. "
        "If the user refers to 'tomorrow', 'yesterday', or similar, resolve it relative to today. "
        "Do not invent future information."
    )


In [ ]:
print("Assistant: Hello! I'm your life assistant. What can I help you with today?")

messages = [
    {"role": "system", "content": (
        "You are a smart, agentic AI assistant with access to tools for Google Calendar, Gmail, Tasks, and Drive. "
        "For simple requests, just collect any missing info, confirm risky actions, and execute. "
        "For complex multi-step goals (like project or study planning), break it into a natural plan and guide the user step-by-step, but do not show subtasks unless the request is complex. "
        "Always be clear, concise, and conversational. Only ask for confirmation before sending, deleting, or sharing things."
    )}
]

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Assistant: Goodbye! Have a great day!")
        break
    if re.search(r'\b(what dates|what did you assign|show.*plan)\b', user_input, re.I):
        show_last_planned_schedule()
        continue
    messages.append({"role": "user", "content": user_input})

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    msg = response.choices[0].message

    if msg.tool_calls:
        for tool_call in msg.tool_calls:
            tool_name = tool_call.function.name
            params = json.loads(tool_call.function.arguments)

            # --- Google Calendar ---
            if tool_name == "create_calendar_event":
                print(f"\n--- CALENDAR EVENT PREVIEW ---")
                print(f"Title: {params['title']}\nStart: {params['start_time']}\nEnd: {params['end_time']}")
                print(f"Location: {params.get('location', '')}\nAttendees: {params.get('attendees', [])}")
                if get_confirmation("Schedule this event? (yes/no): "):
                    result = create_calendar_event(**params)
                else:
                    result = "Event cancelled."

            elif tool_name == "edit_calendar_event":
                result = edit_calendar_event(**params)
            elif tool_name == "cancel_calendar_event":
                if get_confirmation("Are you sure you want to CANCEL this event? (yes/no): "):
                    result = cancel_calendar_event(**params)
                else:
                    result = "Cancel event aborted."
            elif tool_name == "search_calendar_events":
                result = search_calendar_events(**params)

            elif tool_name == "plan_tasks_around_calendar":
                # Delegate preview/confirmation/output to function
                result = plan_tasks_around_calendar(**params)
                print("Assistant:", result)
                messages.append({"role": "assistant", "content": result})
                continue

            # --- Gmail ---
            elif tool_name == "send_gmail":
                print(f"\n--- EMAIL PREVIEW ---")
                print(f"To: {params['to']}\nSubject: {params['subject']}\nStyle: {params.get('style', 'formal')}\nBody:\n{params['body']}")
                if get_confirmation("Send this email? (yes/no): "):
                    result = send_gmail(params['to'], params['subject'], params['body'], params.get('style', 'formal'))
                else:
                    result = "Email cancelled."

            elif tool_name == "search_gmail":
                result = search_gmail(**params)
            elif tool_name == "search_gmail_smart":
                result = search_gmail_smart(**params)
            elif tool_name == "count_gmail_smart":
                result = count_gmail_smart(**params)
            elif tool_name == "read_docx_from_drive":
                result = read_docx_from_drive(**params)
            elif tool_name == "edit_docx_on_drive":
                # Show the old content before edit
                old_text = read_docx_from_drive(params["file_id"])
                print(f"Current content preview:\n{old_text[:1000]}")
                print(f"Proposed new content:\n{params['new_text'][:1000]}")
                confirm = input("Replace doc content with this text? (yes/no): ").strip().lower()
                if confirm == "yes":
                    result = edit_docx_on_drive(**params)
                else:
                    result = "Edit cancelled."
            elif tool_name == "create_google_doc":
                print(f"Title: {params['title']}\nContent Preview: {params['content'][:1000]}")
                confirm = input("Create this Google Doc? (yes/no): ").strip().lower()
                if confirm == "yes":
                    result = create_google_doc(**params)
                else:
                    result = "Doc creation cancelled."
            elif tool_name == "read_gdoc_from_drive":
                result = read_gdoc_from_drive(**params)
                messages.append({"role": "assistant", "content": result})

            elif tool_name == "summarize":
                summary = summarize_text(params['text'])
                messages.append({"role": "assistant", "content": result[:1500]})
            # --- Google Tasks ---
            elif tool_name == "add_google_task":
                result = add_google_task(**params)
            elif tool_name == "edit_google_task":
                print(f"\n--- TASK EDIT PREVIEW ---")
                print(f"Task ID: {params.get('task_id')}")
                print(f"Updates: Title={params.get('title')}, Notes={params.get('notes')}, Due={params.get('due')}, Status={params.get('status')}")
                if get_confirmation("Apply these changes? (yes/no): "):
                    result = edit_google_task(**params)
                else:
                    result = "Edit cancelled."
            elif tool_name == "delete_google_task":
                print(f"\n--- DELETE TASK PREVIEW ---")
                print(f"Task ID: {params.get('task_id')}")
                if get_confirmation("Are you sure you want to DELETE this task? (yes/no): "):
                    result = delete_google_task(params['task_id'])
                else:
                    result = "Delete cancelled."
            elif tool_name == "search_google_tasks":
                result = search_google_tasks(**params)

            # --- Google Drive ---
            elif tool_name == "search_drive_files":
                result = search_drive_files(**params)
                if isinstance(result, list):
                    if not result:
                        result_msg = "No files found in your Drive matching your query."
                        print("Assistant:", result_msg)
                        messages.append({"role": "assistant", "content": result_msg})
                    else:
                        formatted = "\n".join(
                            [f"{i+1}. {f['name']} (open: {f['url']})" for i, f in enumerate(result)]
                        )
                        result_msg = f"I found the following files in your Drive:\n{formatted}"
                        print("Assistant:", result_msg)
                        messages.append({"role": "assistant", "content": result_msg})
                else:
                    result_msg = str(result)
                    print("Assistant:", result_msg)
                    messages.append({"role": "assistant", "content": result_msg})
                continue

            elif tool_name == "drive_file_action":
                action = params.get('action')
                risky = ["delete", "share", "move", "rename"]
                if action in risky:
                    print(f"\n--- DRIVE ACTION PREVIEW ---")
                    print(f"Action: {action}")
                    print(f"File ID: {params.get('file_id', '')}")
                    if not get_confirmation(f"{action.title()} this file? (yes/no): "):
                        result = f"{action.title()} cancelled."
                        print("Assistant:", result)
                        messages.append({"role": "assistant", "content": result})
                        continue
                result = drive_file_action(**params)
                print("Assistant:", result)
                messages.append({"role": "assistant", "content": result})

            else:
                result = "Tool not implemented."

            # Output results and keep chat log
            if isinstance(result, str):
                print("Assistant:", result)
                messages.append({"role": "assistant", "content": result})
            elif isinstance(result, list):
                formatted = "\n".join(
                    [f"{i+1}. {f['name']} (open: {f['url']})" for i, f in enumerate(result)]
                )
                print("Assistant:", formatted)
                messages.append({"role": "assistant", "content": formatted})
            else:
                result_str = str(result)
                print("Assistant:", result_str)
                messages.append({"role": "assistant", "content": result_str})

    else:
        # Natural, conversational LLM reply (no tool call)
        print("Assistant:", msg.content)
        messages.append({"role": "assistant", "content": msg.content})


Assistant: Hello! I'm your life assistant. What can I help you with today?


You:   need to prepare for my exam on SVM --20th july


Assistant: To prepare for your exam on Support Vector Machines (SVM) by July 20th, let's create a structured study plan. Here's a suggested approach:

1. **Understanding SVM Basics:**
   - Study the core concepts of SVM, including the idea of a margin and support vectors.
   - Learn about the kernel trick and how it's used in SVM.

2. **Mathematics Behind SVM:**
   - Review the mathematical formulation of SVM and understand concepts like Lagrange multipliers.
   - Study the dual problem and how it relates to the primal problem in SVM.
  
3. **Practical Applications:**
   - Look into real-world applications of SVM.
   - Work on a few practical problems or case studies.
  
4. **SVM with Different Kernels:**
   - Explore how different kernels (linear, polynomial, RBF) affect the decision boundaries.
  
5. **Advanced Topics:**
   - Delve into multiclass SVM, soft margin classification, and some optimization techniques.

6. **Practice and Review:**
   - Solve past exam papers and practice w

You:  should be good.can you add these as tasks?


Assistant: Task 'Study SVM Basics' added to your Google Tasks.
Assistant: Task 'Review SVM Mathematics' added to your Google Tasks.
Assistant: Task 'Explore SVM Practical Applications' added to your Google Tasks.
Assistant: Task 'Study SVM with Different Kernels' added to your Google Tasks.
Assistant: Task 'Delve into Advanced SVM Topics' added to your Google Tasks.
Assistant: Task 'Solve SVM Practice Problems and Review' added to your Google Tasks.

Assistant: Here’s your suggested plan:
1. Review Introduction to SVM: Linear vs. Non-linear @ 2025-07-08T16:00:00+00:00
2. Study Hyperplanes and Margins @ 2025-07-09T08:00:00+00:00
3. Learn about Kernel Functions @ 2025-07-10T08:00:00+00:00
4. Understand Training: Soft and Hard Margin @ 2025-07-11T08:00:00+00:00
5. Review Parameter Tuning and Optimization @ 2025-07-12T08:00:00+00:00
6. Explore Applications and Examples @ 2025-07-13T08:00:00+00:00
7. Practice Problems @ 2025-07-14T08:00:00+00:00
8. Review Notes @ 2025-07-15T08:00:00+00:00
9

You:  quit
Assign these as Google Tasks with due dates? (yes/no):  no


# UI 

In [ ]:
import gradio as gr
import re
import json
today = get_today_string()
today
messages = [{
    "role": "system",
    "content": (
        f"Today is {today}. "
        "You are a smart, agentic AI assistant with access to tools for Google Calendar, Gmail, Tasks, and Drive. "
        "For simple requests, just collect any missing info, confirm risky actions, and execute. "
        "For complex multi-step goals (like project or study planning), break it into a natural plan and guide the user step-by-step, but do not show subtasks unless the request is complex. "
        "Always be clear, concise, and conversational. Only ask for confirmation before sending, deleting, or sharing things."
    )
}]

# DO NOT redefine your tool functions or LLM client.
# Just use the exact functions you have already defined in your notebook:
# create_calendar_event, edit_calendar_event, cancel_calendar_event,
# plan_tasks_around_calendar, send_gmail, edit_docx_on_drive,
# create_google_doc, edit_google_task, delete_google_task, drive_file_action,
# show_last_planned_schedule, client, tools, etc.

def agentic_chat(messages, pending_action, user_input):
    # Handle confirmation YES/NO
    if user_input.strip().lower() == "yes" and pending_action is not None:
        tool_name = pending_action["tool"]
        params = pending_action["params"]
        if tool_name == "create_calendar_event":
            result = create_calendar_event(**params)
        elif tool_name == "edit_calendar_event":
            result = edit_calendar_event(**params)
        elif tool_name == "cancel_calendar_event":
            result = cancel_calendar_event(**params)
        elif tool_name == "plan_tasks_around_calendar":
            result = plan_tasks_around_calendar(**params)
        elif tool_name == "send_gmail":
            result = send_gmail(params['to'], params['subject'], params['body'], params.get('style', 'formal'))
        elif tool_name == "edit_docx_on_drive":
            result = edit_docx_on_drive(**params)
        elif tool_name == "create_google_doc":
            result = create_google_doc(**params)
        elif tool_name == "add_google_task":
            result = add_google_task(**params)
        elif tool_name == "edit_google_task":
            result = edit_google_task(**params)
        elif tool_name == "delete_google_task":
            result = delete_google_task(params['task_id'])
        elif tool_name == "drive_file_action":
            result = drive_file_action(**params)
        else:
            result = "Tool not implemented."
        messages.append({"role": "assistant", "content": str(result)})
        return messages, None, ""
    elif user_input.strip().lower() == "no" and pending_action is not None:
        messages.append({"role": "assistant", "content": "Okay, cancelled."})
        return messages, None, ""

    # Exit logic
    if user_input.lower() in ["exit", "quit", "bye"]:
        messages.append({"role": "assistant", "content": "Goodbye! Have a great day!"})
        return messages, None, ""

    # Show Plan shortcut
    if re.search(r'\b(what dates|what did you assign|show.*plan)\b', user_input, re.I):
        result = show_last_planned_schedule()
        messages.append({"role": "assistant", "content": result})
        return messages, None, ""

    messages.append({"role": "user", "content": user_input})

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    msg = response.choices[0].message

    # Tool calls (preview & confirmation logic)
    if hasattr(msg, "tool_calls") and msg.tool_calls:
        for tool_call in msg.tool_calls:
            tool_name = tool_call.function.name
            params = json.loads(tool_call.function.arguments)

            if tool_name == "create_calendar_event":
                preview = (
                    f"--- CALENDAR EVENT PREVIEW ---\n"
                    f"Title: {params['title']}\nStart: {params['start_time']}\nEnd: {params['end_time']}\n"
                    f"Location: {params.get('location', '')}\nAttendees: {params.get('attendees', [])}\n"
                    "Schedule this event? (yes/no)"
                )
                messages.append({"role": "assistant", "content": preview})
                return messages, {"tool": tool_name, "params": params}, ""
            elif tool_name == "edit_calendar_event":
                result = edit_calendar_event(**params)
                messages.append({"role": "assistant", "content": result})
            elif tool_name == "cancel_calendar_event":
                preview = "Are you sure you want to CANCEL this event? (yes/no)"
                messages.append({"role": "assistant", "content": preview})
                return messages, {"tool": tool_name, "params": params}, ""
            elif tool_name == "search_calendar_events":
                result = search_calendar_events(**params)
                messages.append({"role": "assistant", "content": result})
            elif tool_name == "plan_tasks_around_calendar":
                preview = "Plan tasks around calendar now? (yes/no)"
                messages.append({"role": "assistant", "content": preview})
                return messages, {"tool": tool_name, "params": params}, ""
            elif tool_name == "send_gmail":
                preview = (
                    f"--- EMAIL PREVIEW ---\n"
                    f"To: {params['to']}\nSubject: {params['subject']}\nStyle: {params.get('style', 'formal')}\nBody:\n{params['body']}\n"
                    "Send this email? (yes/no)"
                )
                messages.append({"role": "assistant", "content": preview})
                return messages, {"tool": tool_name, "params": params}, ""
            elif tool_name == "search_gmail":
                result = search_gmail(**params)
                messages.append({"role": "assistant", "content": result})
            elif tool_name == "search_gmail_smart":
                result = search_gmail_smart(**params)
                messages.append({"role": "assistant", "content": result})
            elif tool_name == "count_gmail_smart":
                result = count_gmail_smart(**params)
                messages.append({"role": "assistant", "content": result})
            elif tool_name == "read_docx_from_drive":
                result = read_docx_from_drive(**params)
                messages.append({"role": "assistant", "content": result})
            elif tool_name == "edit_docx_on_drive":
                old_text = read_docx_from_drive(params["file_id"])
                preview = (
                    f"Current content preview:\n{old_text[:1000]}\n"
                    f"Proposed new content:\n{params['new_text'][:1000]}\n"
                    "Replace doc content with this text? (yes/no)"
                )
                messages.append({"role": "assistant", "content": preview})
                return messages, {"tool": tool_name, "params": params}, ""
            elif tool_name == "create_google_doc":
                preview = (
                    f"Title: {params['title']}\nContent Preview: {params['content'][:1000]}\n"
                    "Create this Google Doc? (yes/no)"
                )
                messages.append({"role": "assistant", "content": preview})
                return messages, {"tool": tool_name, "params": params}, ""
            elif tool_name == "read_gdoc_from_drive":
                result = read_gdoc_from_drive(**params)
                messages.append({"role": "assistant", "content": result})
            elif tool_name == "summarize":
                summary = summarize_text(params['text'])
                messages.append({"role": "assistant", "content": summary[:1500]})
            elif tool_name == "add_google_task":
                preview = f"Task: {params['title']}\nAdd this task? (yes/no)"
                messages.append({"role": "assistant", "content": preview})
                return messages, {"tool": tool_name, "params": params}, ""
            elif tool_name == "edit_google_task":
                preview = (
                    f"--- TASK EDIT PREVIEW ---\n"
                    f"Task ID: {params.get('task_id')}\n"
                    f"Updates: Title={params.get('title')}, Notes={params.get('notes')}, Due={params.get('due')}, Status={params.get('status')}\n"
                    "Apply these changes? (yes/no)"
                )
                messages.append({"role": "assistant", "content": preview})
                return messages, {"tool": tool_name, "params": params}, ""
            elif tool_name == "delete_google_task":
                preview = (
                    f"--- DELETE TASK PREVIEW ---\n"
                    f"Task ID: {params.get('task_id')}\n"
                    "Are you sure you want to DELETE this task? (yes/no)"
                )
                messages.append({"role": "assistant", "content": preview})
                return messages, {"tool": tool_name, "params": params}, ""
            elif tool_name == "search_google_tasks":
                result = search_google_tasks(**params)
                messages.append({"role": "assistant", "content": result})
            elif tool_name == "search_drive_files":
                result = search_drive_files(**params)
                if isinstance(result, list):
                    if not result:
                        result_msg = "No files found in your Drive matching your query."
                    else:
                        formatted = "\n".join(
                            [f"{i+1}. [{f['name']}]({f['url']})" for i, f in enumerate(result)]
                        )
                        result_msg = f"I found the following files in your Drive:\n{formatted}"
                    messages.append({"role": "assistant", "content": result_msg})
                else:
                    messages.append({"role": "assistant", "content": str(result)})
            elif tool_name == "drive_file_action":
                action = params.get('action')
                risky = ["delete", "share", "move", "rename"]
                if action in risky:
                    preview = (
                        f"--- DRIVE ACTION PREVIEW ---\n"
                        f"Action: {action}\nFile ID: {params.get('file_id', '')}\n"
                        f"{action.title()} this file? (yes/no)"
                    )
                    messages.append({"role": "assistant", "content": preview})
                    return messages, {"tool": tool_name, "params": params}, ""
                result = drive_file_action(**params)
                messages.append({"role": "assistant", "content": result})
            else:
                messages.append({"role": "assistant", "content": "Tool not implemented."})

    else:
        messages.append({"role": "assistant", "content": msg.content})

    return messages, None, ""


In [194]:

import gradio as gr
import re, json

# -------- CSS for elegant glassy chat --------
desktop_css = """
body, .gradio-container {
    background: linear-gradient(120deg, #16203a 0%, #29577b 100%) !important;
}
#mainpanel {
    background: rgba(25,34,54,0.73);
    box-shadow: 0 12px 40px 0 rgba(16,29,49,0.18);
    border-radius: 28px;
    margin: 44px auto 0 auto;
    padding: 0 0 34px 0;
    max-width: 900px;
    backdrop-filter: blur(10px);
}
.gr-chat, .gradio-chat {
    background: rgba(26,36,54,0.60) !important;
    border-radius: 22px;
    box-shadow: 0 2px 24px 0 rgba(18,34,69,0.06);
    padding: 20px 0 18px 0;
    backdrop-filter: blur(5px);
}
#ss-title {
    font-size: 2.2em;
    text-align: center;
    font-weight: bold;
    color: #40d0ff;
    letter-spacing: 0.025em;
    margin-top: 20px;
    margin-bottom: 0.17em;
    text-shadow: 0 3px 10px #0b2038aa;
}
#ss-tagline {
    text-align: center;
    font-size: 1.10em;
    color: #b8eaff;
    margin-bottom: 2.2em;
    margin-top: -7px;
    letter-spacing: 0.01em;
}
.gr-message-user {
    background: linear-gradient(90deg,#57acec 70%,#a7e6fd 100%) !important;
    color: #18405b !important;
    font-size: 1.13em;
    border-radius: 19px 19px 7em 20px !important;
    margin: 0.21em 0;
    padding: 15px 26px !important;
    max-width: 76%;
    box-shadow: 0 2px 12px 0 rgba(80,180,255,0.16);
}

.gr-message-assistant {
    background: linear-gradient(90deg,#77e6f7 50%,#e2faff 120%) !important;
    color: #123046 !important;
    font-size: 1.13em;
    border-radius: 19px 19px 20px 7em !important;
    margin: 0.21em 0;
    padding: 15px 26px !important;
    max-width: 76%;
    box-shadow: 0 2px 12px 0 rgba(80,255,255,0.13);
}

.gr-input, .gr-text-input {
    background: rgba(32,49,73,0.93) !important;
    color: #d6f7ff !important;
    font-size: 1.11em !important;
    border-radius: 1.7em !important;
    border: 1.5px solid #2d91bd !important;
    padding: 13px 23px !important;
}
.gr-button {
    background: linear-gradient(90deg, #2caee4 60%, #40d0ff 100%) !important;
    color: #174561 !important;
    font-size: 1.13em !important;
    border-radius: 2em !important;
    font-weight: 500;
    min-width: 104px;
    min-height: 43px;
}
.gr-chatbot, .gr-chatbot-box, .gradio-chatbot {
    background: rgba(25,32,44,0.72) !important;    /* glassy navy-blue */
    border-radius: 22px !important;
    box-shadow: 0 4px 26px 0 rgba(20,36,64,0.11) !important;
    backdrop-filter: blur(7px) !important;
    border: none !important;
}

.gr-chatbot-message, .gr-message {
    background: rgba(38,51,73,0.16) !important;   /* faint overlay behind bubbles */
}
.gr-chatbot-message, .gr-message {
    box-shadow: none !important;
    border: none !important;
}

::-webkit-scrollbar {width: 10px; background: #172c46;}
::-webkit-scrollbar-thumb {background: #40d0ff; border-radius: 1em;}
"""


# --------- GRADIO UI (desktop, glassy, elegant) ---------
with gr.Blocks(css=desktop_css) as demo:
    with gr.Column(elem_id="mainpanel"):
        gr.Markdown('<div id="ss-title">SmartSphere AI</div>')
        gr.Markdown('<div id="ss-tagline">Seamless support, smarter living.</div>')
        chatbot = gr.Chatbot(type="messages", height=430, show_label=False)
        user_txt = gr.Textbox(placeholder="Type your message...", show_label=False)
        state = gr.State([{"role": "assistant", "content": "Hello! I'm your life assistant. What can I help you with today?"}])
        pending_action_state = gr.State(None)
        def respond(messages, pending_action, user_txt):
            new_messages, new_pending, txt = agentic_chat(messages, pending_action, user_txt)
            return new_messages, new_messages, new_pending, ""

        user_txt.submit(respond, [state, pending_action_state, user_txt], [chatbot, state, pending_action_state, user_txt])
        demo.load(lambda: ([{"role": "assistant", "content": "Hello! I'm your life assistant. What can I help you with today?"}], 
                          [{"role": "assistant", "content": "Hello! I'm your life assistant. What can I help you with today?"}], None, ""), 
                  outputs=[chatbot, state, pending_action_state, user_txt])


   # send_btn.click(agentic_chat, inputs=[state, txt], outputs=[chatbot, txt, state])
#    txt.submit(agentic_chat, inputs=[state, txt], outputs=[chatbot, txt, state])

demo.launch()



* Running on local URL:  http://127.0.0.1:7908
* To create a public link, set `share=True` in `launch()`.
